In [2]:
import pandas as pd


import pickle
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score

In [3]:
data = pd.read_csv('tutorial_sample_data.csv')

In [4]:
data.head()

,x,xx,xxx,xxxx,xxxxx,level_1,level_2
0,23,42,17,36,23,A,X
1,40,11,2,41,36,A,X
2,50,32,12,21,25,A,X
3,49,31,48,17,49,A,Y
4,9,33,24,39,4,A,Y


In [5]:
train, test = train_test_split(data, test_size=0.2, random_state=1811)


In [6]:
train.shape
test.shape

(10, 7)

In [7]:
train.head()
# Notice we have more data than we want right now
# as we do not want the level1 or level 2 cols being used by the models

,x,xx,xxx,xxxx,xxxxx,level_1,level_2
6,21,22,21,50,12,A,Y
42,17,23,23,37,28,C,G
14,41,15,41,14,6,B,P
30,33,20,19,34,2,B,R
40,3,50,4,1,7,C,G


In [8]:
class1 = train['level_1'].astype(str)
class2 = train['level_2'].astype(str)

In [9]:
%%time
# create model
tree = DecisionTreeClassifier(max_depth=5)

# train or fit the model on the level 1 classifications using the data from the information cols.
tree.fit(train[['x', 'xx','xxx','xxxx','xxxxx']], class1)


Wall time: 5.04 ms


DecisionTreeClassifier(max_depth=5)

In [10]:
# save level 1 classifier
with open('level1.pk', 'wb') as cls:
    pickle.dump(tree, cls)

In [11]:
# in this space you will be fitting models for the level 2 data 
#so you will want to split the test data into sub data for each classification e.g.

x = list(class1[class1 == 'A'].index)

print('This is a list of the indexes for when the train set has a level 1 class of A')
x

This is a list of the indexes for when the train set has a level 1 class of A


[6, 3, 5, 0, 7, 9]

In [20]:
train.loc[x,['x', 'xx','xxx','xxxx','xxxxx']]
train.loc[x, :]

# returns all the data for which is level 1 class of A

,x,xx,xxx,xxxx,xxxxx,level_1,level_2
6,21,22,21,50,12,A,Y
3,49,31,48,17,49,A,Y
5,44,45,31,13,47,A,Y
0,23,42,17,36,23,A,X
7,47,18,1,44,3,A,Z
9,19,16,50,35,27,A,Z


In [13]:
class2[x]

#returns all of the Level 2 data for which level 1 class is A

6    Y
3    Y
5    Y
0    X
7    Z
9    Z
Name: level_2, dtype: object

In [14]:
%%time
# run another model 
tree = DecisionTreeClassifier(max_depth=5)
#running model on all of train data for which level 1 class is A on level 2 where lvl1 class = A
tree.fit(train.loc[x,['x', 'xx','xxx','xxxx','xxxxx']], class2[x])

# save level 2A classifier
with open('level2_A.pk', 'wb') as cls:
    pickle.dump(tree, cls)

Wall time: 3.99 ms


In [15]:
# load the model
with open('level1.pk', 'rb') as nb:
    model = pickle.load(nb)

In [16]:
# use the loaded model to make predictions

level1_pred = model.predict(test[['x', 'xx','xxx','xxxx','xxxxx']])

In [17]:
# take a look at the predictions - note only 10 as that is test set size.
level1_pred

array(['B', 'C', 'A', 'C', 'A', 'B', 'C', 'C', 'C', 'C'], dtype=object)

In [18]:
# use accuracy_score function to get the scores form test set acutal to the predictions the model made 
accuracy_score(test[['level_1']], level1_pred)

0.5